# 英语句子的语义

## 基于特征的文法中的合成语义学

前面我们在句法分析的基础上构建了一个 SQL 查询，现在我们将建立一个逻辑形式的语义表示方法。我们设计这样的文法的指导思想之一是**组合原则**（Frege 原则），即整体的含义是其各部分的含义以及它们的句法组合方式的一个函数。

我们现在的目标是以一种可以与分析过程平滑对接的方式整合语义表达的构建，如下图所示：

![ch10-tree-3.png](resources/ch10-tree-3.png)

根节点的 SEM 值显示了整个句子的语义表示，而较低节点的 SEM 值显示句子成分的语义表示。我们把 SEM 值放在尖括号中来与其他特征值区别开。

我们该如何编写能给我们这样的结果的文法规则呢？与前面章节的文法类似，我们首先为词汇节点指定语义表示，然后租着它们的子节点的每个部分的语义表示。不同的是，我们将使用**函数应用**而不是**字符串连接**作为组成的模式（这里我们省略了尖括号）：

    S[SEM=<?vp(?np)>] -> NP[SEM=?np] VP[SEM=?vp]
    
这个文法规则表明：给定某个 SEM 值 ?np 表示主语 NP，某个 SEM 值 ?vp 表示 VP，那么父节点 S 的 SEM 值将通过 ?vp(?np) 这样的函数表达式来构建。

我们添加以下规则来完成整个文法：

    VP[SEM=?v] -> IV[SEM=?v]
    NP[SEM=<cyril>] -> 'Cyril'
    IV[SEM=<\x.bark(x)>] -> 'barks'
    
这里 barks 规则中有一串额外的符号，后面将做更详细的解释。接下来我们介绍一种新的工具，称为 λ 演算，它可以在我们组装一个英文句子的语义表示时组合一阶逻辑表达式。

## λ 演算

数学集合符合对于制定我们想从文档中选择的词的属性 P 很有用，我们用下面的公式来表示“所有 w 的集合，其中 w 是 V（词汇表）的元素且 w 具有属性 P”：

    {w | w ∈ V & P(w)}
    
使用 **[λ 运算符](https://zh.wikipedia.org/wiki/%CE%9B%E6%BC%94%E7%AE%97)** 可以表达同样的意思，其中 λ 我们可以认为是一个约束运算符，类似于一阶逻辑量词：

    λw. (V(w) ∧ P(w))
    
在 NLTK 中我们用斜杠来表示 λ 运算符，下面例子中的 a 为开放公式，b 为其对应的 λ 表达式，c 为 NLTK 中的 λ 表达式：

    a. (walk(x) ∧ chew_gum(x))
    b. λx.(walk(x) ∧ chew_gum(x))
    c. \x.(walk(x) & chew_gum(x))

In [1]:
import nltk
read_expr = nltk.sem.Expression.fromstring
expr = read_expr(r'\x.(walk(x) & chew_gum(x))')
print(expr)
print(type(expr))
print(expr.free())

\x.(walk(x) & chew_gum(x))
<class 'nltk.sem.logic.LambdaExpression'>
set()


我们对在表达式中绑定变量给出了一个特殊的名字：λ 抽象。上面例子的英语表示是“是一个 x，其中 x 步行且 x 嚼口香糖”或“具有步行和嚼口香糖的属性”，通常认为 λ 抽象可以很好地表示动词短语（或无主语从句），如下面的例子所示：

    a. To walk and chew-gum is hard
    b. hard(\x.(walk(x) & chew_gum(x)))
    
一般而言，如果一个给定的开放公式 φ 中有自由变量 x，那么 x 的 λ 抽象表达式为 λx.φ —— 即满足 φ 的 x。如果 φ 是类型 τ，而 x 是类型 e 的变量，那么 λx.φ 就是 <e,τ> 类型的。

λx.φ 类似于**一元谓词**，我们可以替换其中的变量为特定的实体，例如：

    a. \x.(walk(x) & chew_gum(x)) (gerald)
    b. (walk(gerald) & chew_gum(gerald))
    
从公式 a 到公式 b 的约简是简化语义表示的非常有用的操作，通常被称为 β-约简，在 NLTK 中可以通过调用 simplify() 方法来实现表达式的 β-约简：

In [2]:
expr = read_expr(r'\x.(walk(x) & chew_gum(x))(gerald)')
print(expr)
print(expr.simplify())

\x.(walk(x) & chew_gum(x))(gerald)
(walk(gerald) & chew_gum(gerald))


虽然我们至今只考虑了 λ 抽象的主体是一个某种类型 τ 的开放公式，但这并不是必要的限制，主体可以是任何符合文法的表达式，下面是一个有两个 λ 的例子：
    
    \x.\y.(dog(x) & own(y, x))
    
如果说前面的 λ 抽象起到一元谓词的作用，那么这个 λ 抽象就像一个**二元谓词**，它可以直接应用到两个参数。在 NLTK 中，嵌套的 λ 如 \x.\y. 可以缩写为 \x y：

In [3]:
print(read_expr(r'\x.\y.(dog(x) & own(y, x))(cyril)').simplify())
print(read_expr(r'\x y.(dog(x) & own(y, x))(cyril, angus)').simplify())

\y.(dog(cyril) & own(y,cyril))
(dog(cyril) & own(angus,cyril))


我们所有的 λ 抽象到目前为止都只设计熟悉的一阶变量 x、y 等，它们都是类型 e 的变量，但假设我们要处理一个抽象，例如 \x.walk(x) 作为另一个 λ 抽象 的参数呢？
    
    \y.y(angus)(\x.walk(x))
    
由于自由变量 y 规定是 e 类型的，\y.y(angus) 只适用于 e 类型的参数，而 \x.walk(x) 是 <e,τ> 类型的，因此我们需要允许在更高级的类型的变量上进行抽象。我们用大写字母 P 和 Q 等表示类型为 <e,τ> 的参数，那么抽象 \P.P(angus) 就是 <<e,τ>,τ> 类型的，这样 \P.P(angus)(\x.walk(x)) 就是合法的，可以通过 β-约简化简为 \x.walk(x)(angus)，然后再次化简为 walk(angus)。

在进行 β-约简时，对变量有些注意事项，例如下面的公式 a 和 b 中的 x 和 z 只是一个自由变量的不同身份：

    a. \y.see(y, x)
    b. \y.see(y, z)
    
限制假设我们应用 \P.exists x.P(x) 到这两个公式：

    a. \P.exists x.P(x)(\y.see(y, x))
    b. \P.exists x.P(x)(\y.see(y, z))
    
进行 β-约简后，我们得到了不同的结果：
    
    a. exists x.see(x, x)
    b. exists x.see(x, z)
    
其中结果 a 是指有某个 x 能看到自己，而结果 b 的意思是某个 x 能看到一个未指定的个体 z，显然第一种结果是错误的，我们要禁止这种**变量捕获**。事实上，给定任何绑定变量的表达式（包括 ∀，∃ 或 λ），未绑定的变量选择名字完全是任意的，例如 exists x.P(x) 和 exists y.P(y) 是等价的，它们被称为 **α-等价**，或**字母变体**。重新标记绑定的变量的过程被称为 **α-转换**，当我们在 NLTK 中使用 == 符号来测试不同的 VariableBinderExpressions 是否相等时，我们其实就是在测试它们是否 α-等价：

In [4]:
expr1 = read_expr('exists x.P(x)')
print(expr1)
expr2 = expr1.alpha_convert(nltk.sem.Variable('z'))
print(expr2)
print(expr1 == expr2)

exists x.P(x)
exists z.P(z)
True


当我们对一个应用 f(a) 进行β-约简时，首先会检查是否有自然变量在 a 中同时也作为 f 的子术语中绑定的变量出现，例如在上面讨论的例子中，x 是 a 的自由变量，f 包括子术语 exists x.P(x)。在这种情况下，我们产生一个 exists x.P(x) 的字母变体，即 exists z1.P(z1)，然后再进行约简。这种重新标记在 simplify() 方法中是自动进行的：

In [5]:
expr3 = read_expr(r'\P.(exists x.P(x))(\y.see(y, x))')
print(expr3)
print(expr3.simplify())

(\P.exists x.P(x))(\y.see(y,x))
exists z1.see(z1,x)


## 量化的 NP

在本节开始，我们简要介绍了如何为 Cyril barks 构建语义表示。那么对于量化的 NP 要如何处理呢？例如：

    a. A dog barks.
    b. exists x.(dog(x) & bark(x))
    
按照前面的语法规则，我们可以得到最终的语句的 SEM 值为 [SEM=<?np(\x.bark(x))>]，将 ?np 替换为什么可以使其等价于 [SEM=<exists x.(dog(x) & bark(x))>] 呢？联想到前面的 λ 抽象和 β-约简，我们可以用下面的 λ 抽象来表示 ?np，这里的 P 是类型为 <e,τ> 的函数表达式：

    \P.exists x.(dog(x) & P(x))
    
然后我们再做进一步的抽象，加上将限定词 a 和 dog 的语义组合的过程：

    \Q P.exists x.(Q(x) & P(x))
    
对于这一抽象，我们先应用 \x.dog(x)，再应用 \x.bark(x)，就可以进行 β-约简产生我们想要的结果，即 exists x.(dog(x) & bark(x))：

In [6]:
expr4 = read_expr(r'\Q P.(exists x.(Q(x) & P(x)))(\x.dog(x), \x.bark(x))')
print(expr4)
print(expr4.simplify())

((\Q P.exists x.(Q(x) & P(x)))(\x.dog(x)))(\x.bark(x))
exists x.(dog(x) & bark(x))


## 及物动词

我们的下一个挑战是处理含及物动词的句子，例如：

    a. Angus chases a dog.
    b. exists x.(dog(x) & chase(angus, x))
    
在给出解决方案之前，我们需要提两个重要的制约因素：一是 a dog 的语义表示与 NP 是否充当句子的主语或宾语相互独立，也就是说我们希望能坚持用 \P.exists x.(dog(x) & P(x)) 来作为 a dog 的语义表示；二是 VP 应该有一个同意的解释类型，不管它只是一个不及物动词还是及物动词加对象的组合，也就是说希望 VP 的类型一直是 <e,τ>。基于这些制约因素，chases a dog （去掉了主语）的语义应该表示为：

    \y.exists x.(dog(x) & chase(y, x))

由于 a dog 的语义表示为 \P.exists x.(dog(x) & P(x))，因此我们现在的任务是为 chases 设计语义表示，使得它可以和 a dog 的结合从而派生出 \y.exists x.(dog(x) & chase(y, x)) 的语义表示。我们进行 β-约简的逆操作，可以得到：

    \P.exists x.(dog(x) & P(x))(\z.chase(y, z))
    
可以在 NLTK 中证明它们的等价性：

In [7]:
expr5 = read_expr(r'\P.(exists x.(dog(x) & P(x)))(\z.chase(y, z))')
print(expr5)
print(expr5.simplify())

(\P.exists x.(dog(x) & P(x)))(\z.chase(y,z))
exists x.(dog(x) & chase(y,x))


我们用变量 X 来替换其中的 \P.exists x.(dog(x) & P(x))，就得到了 chases 的语义表达式：

    \X X(\z.chase(y, z))

需要注意的是，我们需要保证该语义表达式与 NP 结合后产生 <e,τ> 类型（VP），故我们稍作修改，加入主语变量 y 的抽象：

    \X y.X(\z.chase(y, z))
    
我们可以对 z 进行字母变体，用 x 来替代：

In [8]:
tvp = read_expr(r'\X x.X(\y.chase(x,y))')
np = read_expr(r'(\P.exists x.(dog(x) & P(x)))')
vp = nltk.sem.ApplicationExpression(tvp, np)
print(vp)
print(vp.simplify())

(\X x.X(\y.chase(x,y)))(\P.exists x.(dog(x) & P(x)))
\x.exists z2.(dog(z2) & chase(x,z2))


完成了 VP 的语义表示，接下来我们需要结合主语 NP 的语义来建立整个句子的语义表示。如果主语 NP 是一个量化的表达式，例如 every girl、a dog 这样，那么就可以和前面一样将主语转换成函数表达式。然而，如果主语是特定的对象时，我们暂时还没有方法将它们当作函数表达式使用。因此我们需要为它们提出新的语义表示，使它们也成为如量化的 NP 那样的函数表达式：

    \P.P(angus)
    
\P.P(angus) 表示相应与 Angus 为真的所有属性的集合的特征函数，我们在其上应用其它函数时，例如 \P.P(angus)(\x.walk(x))，就可以化简为 walk(angus)。

NLTK 中的文法 simple-sem.fcfg 包含了一个用于分析和翻译我们前面用到的例子的规则集合：

In [9]:
from nltk import load_parser
parser = load_parser('grammars/book_grammars/simple-sem.fcfg', trace=0)
sentence = 'Angus gives a bone to every dog'
tokens = sentence.split()
for tree in parser.parse(tokens):
    print(tree.label()['SEM'])

all z4.(dog(z4) -> exists z3.(bone(z3) & give(angus,z3,z4)))


NLTK 提供了一些实用工具帮助我们更方便地获取和检查语义，函数 interpret_sents() 用于批量翻译输入的句子，对于每个句子其结果包含分析树和语义表示组成的 (synrep, semrep) 对：

In [10]:
sents = ['Irene walks', 'Cyril bites an ankle']
grammar_file = 'grammars/book_grammars/simple-sem.fcfg'
for results in nltk.interpret_sents(sents, grammar_file):
    for (synrep, semrep) in results:
        print(synrep)

(S[SEM=<walk(irene)>]
  (NP[-LOC, NUM='sg', SEM=<\P.P(irene)>]
    (PropN[-LOC, NUM='sg', SEM=<\P.P(irene)>] Irene))
  (VP[NUM='sg', SEM=<\x.walk(x)>]
    (IV[NUM='sg', SEM=<\x.walk(x)>, TNS='pres'] walks)))
(S[SEM=<exists z5.(ankle(z5) & bite(cyril,z5))>]
  (NP[-LOC, NUM='sg', SEM=<\P.P(cyril)>]
    (PropN[-LOC, NUM='sg', SEM=<\P.P(cyril)>] Cyril))
  (VP[NUM='sg', SEM=<\x.exists z5.(ankle(z5) & bite(x,z5))>]
    (TV[NUM='sg', SEM=<\X x.X(\y.bite(x,y))>, TNS='pres'] bites)
    (NP[NUM='sg', SEM=<\Q.exists x.(ankle(x) & Q(x))>]
      (Det[NUM='sg', SEM=<\P Q.exists x.(P(x) & Q(x))>] an)
      (Nom[NUM='sg', SEM=<\x.ankle(x)>]
        (N[NUM='sg', SEM=<\x.ankle(x)>] ankle)))))


现在我们知道如何把英文句子翻译成逻辑形式，前面我们有在模型中如何检查逻辑形式的真假，把这两者结合起来，我们就可以检查一个英文句子在给定的模型中是否为真。evaluate_sents() 方法与 interpret_sents() 类似，除了它还需要一个模型和一个变量赋值作为参数，输出中也多了一个 value 用来表明句子的真值：

In [11]:
v = """
    bertie => b
    olive => o
    cyril => c
    boy => {b}
    girl => {o}
    dog => {c}
    walk => {o, c}
    see => {(b, o), (c, b), (o, c)}
"""
val = nltk.Valuation.fromstring(v)
g = nltk.Assignment(val.domain)
m = nltk.Model(val.domain, val)
sent = 'Cyril sees every boy'
grammar_file = 'grammars/book_grammars/simple-sem.fcfg'
results = nltk.evaluate_sents([sent], grammar_file, m, g)[0]
for (syntree, semrep, value) in results:
    print(semrep)
    print(value)

all z6.(boy(z6) -> see(cyril,z6))
True


## 再述量词歧义

上述方法的一个重要限制使它没有处理范围歧义。我们的翻译方法是句法驱动的，认为语义表示与句法分析紧密耦合，语义中量词的范围也因此反映了句法分析树中相应的 NP 的范围。因此对于像 Every girl chases a dog. 这样的句子，总是翻译成下面的公式 a 而不是公式 b：

    a. all x.(girl(x) -> exists y.(dog(y) & chase(x, y)))
    b. exists y.(dog(y) & all x.(girl(x) -> chase(x, y)))
    
有许多方法来处理范围一起，我们简要地看看最简单的一个。首先，让我们考虑具有范围的公式的结构，下图描绘了两种不同的读法：

![quant-ambig.png](resources/quant-ambig.png)

左侧的结构中，在最上方有对于与 every girl 的量词，φ 可以被看作是量词范围内的所有东西的一个占位符，向下移动，我们看到可以插入相应与 a dog 的量词作为 φ 的实例，这提供了一种新的占位符 ψ 表示 a dog 的范围。右侧的结构是相同的，除了两个量词的顺序交换了。

在被称为 **Copper 存储**的方法中，语义表示不再是一阶逻辑表达式，而是由一个**核心**语义表示加一个**绑定操作符**链表组成的配对，可以认为一个**绑定操作符**是量化 NP 的语义表示。以 Every girl chases a dog. 为例，我们将开放公式 chase(x, y) 作为核心，两个 NP 构成绑定操作符列表。我们将一个绑定操作符从链表中挑出来，与核心结合：

    \P.exists y.(dog(y) & P(y))(\z2.chase(z1, z2))
    
然后我们将链表中的另一个绑定操作符应用到结果中：

    \P.all x.(girl(x) -> P(x))(\z1.exists x.(dog(x) & chase(z1, x)))
    
当链表为空时，我们就有了句子的传统逻辑形式，将绑定操作符和核心以这种方式组合被称为 **S-检索**。如果我们允许绑定操作符使用的不同顺序，就能够产生量词的每一个可能的范围排序。

那么我们要如何建立一个**核心 + 存储**表示的组合呢？我们可以为文法中的每个短语和词法规则的 SEM 特征添加属性 CORE 和 STORE。以 Cyril smiles 这个简单的句子为例，下面是动词 smiles 的词法规则：
    
    IV[SEM=[core=<\x.smile(x)>, store=(/)]] -> 'smiles'
    
特定名称 Cyril 的规则更加复杂：

    NP[SEM=[core=<@x>, store=(<bo(\P.P(cyril), @x)>)]] -> 'Cyril'
    
谓词 bo 有两个子部分：一个确定的名称 cyril 的标准表示（类型提升）\P.P(cyril)，以及表达式 @x，它被称为绑定操作符的**地址**。@x 是原变量，也就是范围在逻辑的独立变量之上的变量，它提供了核心 CORE 的值。VP 的规则只是向上渗透 IV 的语义，有趣的工作由 S 规则来完成：

    VP[SEM=?s] -> IV[SEM=?s]
    S[SEM=[core=<?vp(?np)>, store=(?b1+?b2)]] -> 
        NP[SEM=[core=?np, store=?b1]] VP[SEM=[core=?vp, store=?b2]]
        
S 节点的核心值是应用 VP 的核心值，即 \x.smile(x) 到主语 NP 的结果，NP 的核心值并不是 @x，而是一个 @x 的实例，例如 z3。经过 β-约简后，<?vp(?np)> 即为 <smile(z3)>。**需要注意的是，STORE 中出现的 @x 和 CORE 中的 @x 会被映射到同样的值，也就是 z3。**整体的分析树如下：

    (S[SEM=[core=<smile(z3)>, store=(bo(\P.P(cyril), z3))]]
        (NP[SEM=[core=<z3>, store=(bo(\P.P(cyril), z3))]] Cyril)
            (VP[SEM=[core=<\x.smile(x)>, store=()]]
            (IV[SEM=[core=<\x.smile(x)>, store=()]] smiles)))
            
回到开头的例子 Every girl chases a dog，其存储风格的 SEM 值是：
    
    core = <chase(z1, z2)>
    store = (bo(\P.all x.(girl(x) -> P(x)), z1), bo(\P.exists x.(dog(x) & P(x)), z2))
    
现在我们执行 **S-检索**的过程，将绑定操作符从 store 中去除，先后应用到 core。假设我们先用 bo(\P.all x.(girl(x) -> P(x)), z1) 和 chase(z1, z2) 结合，那么后者需要先被转换成 λ-抽象，在那些变量上进行抽象呢，这就是 z1 告诉我们的，也就是抽象成 \z1.chase(z1, z2)。nltk.sem.cooper_storage 模块可以帮助我们实现 Copper 存储：

In [12]:
from nltk.sem import cooper_storage as cs
sentence = 'every girl chases a dog'
trees = cs.parse_with_bindops(sentence, grammar='grammars/book_grammars/storage.fcfg')
semrep = trees[0].label()['SEM']
cs_semrep = cs.CooperStore(semrep)
print(cs_semrep.core)
for bo in cs_semrep.store:
    print(bo)

chase(z2,z3)
bo(\P.all x.(girl(x) -> P(x)),z2)
bo(\P.exists x.(dog(x) & P(x)),z3)


最后我们调用 s_retrieve() 方法检查读法：

In [13]:
cs_semrep.s_retrieve(trace=True)

Permutation 1
   (\P.all x.(girl(x) -> P(x)))(\z2.chase(z2,z3))
   (\P.exists x.(dog(x) & P(x)))(\z3.all x.(girl(x) -> chase(x,z3)))
Permutation 2
   (\P.exists x.(dog(x) & P(x)))(\z3.chase(z2,z3))
   (\P.all x.(girl(x) -> P(x)))(\z2.exists x.(dog(x) & chase(z2,x)))


In [14]:
for reading in cs_semrep.readings:
    print(reading)

exists x.(dog(x) & all z9.(girl(z9) -> chase(z9,x)))
all x.(girl(x) -> exists z10.(dog(z10) & chase(x,z10)))
